In [2]:
!pip install flask-ngrok

  Using cached flask_ngrok-0.0.25-py3-none-any.whl (3.1 kB)
  Using cached Flask-2.0.2-py3-none-any.whl (95 kB)
  Using cached Werkzeug-2.0.2-py3-none-any.whl (288 kB)
  Using cached itsdangerous-2.0.1-py3-none-any.whl (18 kB)


In [1]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request

In [2]:
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import json
import nltk
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Roger\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data = pd.read_csv('dataset.csv')
data[5:7]

,Title,Abstract,Classifications
5,"Value, Variety and Viability: Designing For Co...",While service-dominant logic proposes that all...,THEORETICAL
6,3D U-Net: Learning Dense Volumetric Segmentati...,This paper introduces a network for volumetric...,ENGINEERING


In [4]:
with open('word2idx.json', 'r') as f:
    word2idx = json.loads(f.read())

In [5]:
word2idx.get("<pad>" , word2idx["<unk>"])

0

In [6]:
class simpleNet(nn.Module):
    def __init__(self,vocab_num, vocabulary_size):
        super(simpleNet, self).__init__()
        self.hidden_dim = 512
        self.embedding = nn.Embedding(vocab_num,vocabulary_size,padding_idx=0) 
        self.sent_rnn = nn.GRU(vocabulary_size,
                                self.hidden_dim,
                                bidirectional=True,
                                batch_first=True)
        self.l1 = nn.Linear(self.hidden_dim, 4)

    def forward(self, x):
        x = self.embedding(x)
        b,s,w,e = x.shape
        x = x.view(b,s*w,e)
        x, __ = self.sent_rnn(x)
        x = x.view(b,s,w,-1)
        x = torch.max(x,dim=2)[0]
        x = x[:,:,:self.hidden_dim] + x[:,:,self.hidden_dim:]
        x = torch.max(x,dim=1)[0]
        x = torch.sigmoid(self.l1(F.relu(x)))
        return x

In [7]:
model = simpleNet(len(word2idx),300)
model.load_state_dict(torch.load('model_final.pkl',
                                 map_location=torch.device('cpu')))
model

simpleNet(
  (embedding): Embedding(40169, 300, padding_idx=0)
  (sent_rnn): GRU(300, 512, batch_first=True, bidirectional=True)
  (l1): Linear(in_features=512, out_features=4, bias=True)
)

In [8]:
from nltk.tokenize import word_tokenize

def sentence_to_indices(sentence, word_dict):
  return [word_dict.get(word, word_dict["<unk>"]) for word in word_tokenize(sentence) ]

def preprocess_sample(text, word_dict , max_len = 500):
  processed = [sentence_to_indices(sent, word_dict) for sent in text.split('$$$')]
  max_len = max([min(len(sentence), max_len) for sentence in processed])
  pad_abstract = []
  for sentence in processed:
    if len(sentence) > max_len:
      pad_abstract.append(sentence[:max_len])
    else:
      pad_abstract.append(sentence+[word_dict['<pad>']]*(max_len-len(sentence)))
  return pad_abstract

In [9]:
idx = 12
def Testing(idx):
  print(data['Abstract'][idx])
  sent_p = preprocess_sample(data['Abstract'][idx],word2idx)
  input = torch.LongTensor([sent_p])
  output = model(input).squeeze(0)
  label = ['THEORETICAL', 'ENGINEERING', 'EMPIRICAL', 'OTHERS']
  print('Predition :',end = ' ')
  for i,cls in enumerate(output):
    if cls > 0.5:
      print(label[i],end = ' ')
  print('')
  print('Anwser :',data['Classifications'][idx])

Testing(idx)

In this paper, we present a detailed design of dynamic video segmentation network (DVSNet) for fast and efficient semantic video segmentation.$$$DVSNet consists of two convolutional neural networks: a segmentation network and a flow network.$$$The former generates highly accurate semantic segmentations, but is deeper and slower.$$$The latter is much faster than the former, but its output requires further processing to generate less accurate semantic segmentations.$$$We explore the use of a decision network to adaptively assign different frame regions to different networks based on a metric called expected confidence score.$$$Frame regions with a higher expected confidence score traverse the flow network.$$$Frame regions with a lower expected confidence score have to pass through the segmentation network.$$$We have extensively performed experiments on various configurations of DVSNet, and investigated a number of variants for the proposed decision network.$$$The experimental results sho

In [10]:
data['Abstract'][99]

'We consider the transmission of packets across a lossy end-to-end network path so as to achieve low in-order delivery delay.$$$This can be formulated as a decision problem, namely deciding whether the next packet to send should be an information packet or a coded packet.$$$Importantly, this decision is made based on delayed feedback from the receiver.$$$While an exact solution to this decision problem is challenging, we exploit ideas from queueing theory to derive scheduling policies based on prediction of a receiver queue length that, while suboptimal, can be efficiently implemented and offer substantially better performance than state of the art approaches.$$$We obtain a number of useful analytic bounds that help characterise design trade-offs and our analysis highlights that the use of prediction plays a key role in achieving good performance in the presence of significant feedback delay.$$$Our approach readily generalises to networks of paths and we illustrate this by application 

In [11]:
def get_result(text):
  sent_p = preprocess_sample(text,word2idx)
  input = torch.LongTensor([sent_p])
  output = model(input).squeeze(0)
  label = ['THEORETICAL', 'ENGINEERING', 'EMPIRICAL', 'OTHERS']
  anwser = []
  for i,cls in enumerate(output):
    if cls > 0.5:
      anwser.append(label[i])
  print(anwser)
  return anwser

In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request, url_for, redirect, flash,render_template

TEMPLATE = 'templates'
STATIC = 'static'

app = Flask(__name__,template_folder=TEMPLATE,static_folder=STATIC)
run_with_ngrok(app)

input_text = ''


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0d47-140-127-114-22.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Dec/2021 15:30:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 15:30:29] "GET /static/css/nicepage.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 15:30:29] "GET /static/css/Page-1.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 15:30:29] "GET /static/css/bootstrap/bootstrap.css HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 15:30:29] "GET /static/images/61642b43-817b-ae1b-79ed-23f3c98bee6c.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 15:30:29] "GET /static/images/banner-bg.png HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 15:30:31] "GET /static/images/images-removebg-preview.ico HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 15:37:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2021 15:37:41] "GET /static/css/nicepage.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Dec/2021 15:37:41] "GET /static/css/Page-1.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Dec/2021 15:37:41] "GET /static/css/bootstrap/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Dec/2021 15:37:41] "GET /static/images/6164

['THEORETICAL']
